# commander examples


In [28]:
from typing import List
from math import pi
import rospy
from copy import deepcopy

from moveit_commander import PlanningSceneInterface

from geometry_msgs.msg import (
    Pose,
    PoseStamped,
    Point,
    Quaternion,
)
from commander.msg import Goal
from commander.srv import (
    ExecuteTrajectory,
    PlanGoal,
    PlanGoalRequest,
    PlanSequence,
    PlanSequenceRequest,
    PickPlace,
    GetTcpPose,
    VisualizePoses,
    SetEe,
)

from commander.utils import poses_from_yaml, load_scene
from commander.transform_utils import orient_poses, create_rotation_matrix, apply_transformation
from commander.ur_robot import UrRobot

plan_goal_srv = rospy.ServiceProxy('commander/plan_goal', PlanGoal)
plan_sequence_srv = rospy.ServiceProxy('commander/plan_sequence', PlanSequence)
execute_trajectory_srv = rospy.ServiceProxy('commander/execute_trajectory', ExecuteTrajectory)
get_tcp_pose_srv = rospy.ServiceProxy('commander/get_tcp_pose', GetTcpPose)
set_ee_srv = rospy.ServiceProxy('commander/set_ee', SetEe)
pick_place_srv = rospy.ServiceProxy('commander/pick_place', PickPlace)


def display_poses(poses: List[Pose], frame_id: str = 'base_link') -> None:
    rospy.wait_for_service('/visualize_poses', timeout=10)
    visualize_poses = rospy.ServiceProxy('/visualize_poses', VisualizePoses)
    visualize_poses(frame_id, poses)


rospy.init_node('robot_program')
scene = PlanningSceneInterface()
load_scene()


plan to goal

In [29]:
cam_home = [0.0, -1.5708, 1.5708, -3.1416, -1.5708, 0]


In [30]:
# the following args for planner are available:
# - 'ompl' will use the ompl planning pipeline with RTTConnect planner
# - 'ptp' will use the pilz planning pipeline with the PTP planner
# - 'lin' will use the pilz planning pipeline with the LIN planner

plan_goal_srv(Goal(joint_values=cam_home, vel_scale=0.2, acc_scale=0.2, planner='ptp'))

success: True
configuration_change: True

In [31]:
success = execute_trajectory_srv()


In [32]:
success = set_ee_srv('gripper_tcp')


In [33]:
target0 = Pose(
    position=Point(0.5, -0.4, 0.4),
    orientation=Quaternion(0.0, -0.7071067811865475, -0.7071067811865475, 0.0),
)
display_poses([target0])
success = plan_goal_srv(Goal(pose=target0,
        vel_scale=0.2,
        acc_scale=0.2,
        planner='ptp'))

In [34]:
success = execute_trajectory_srv()

In [ ]:
#Close gripper
UrRobot().set_do(do = 6, state = 0)
UrRobot().set_do(do = 4, state = 1)

In [ ]:
#Open gripper
UrRobot().set_do(do = 4, state = 0)
UrRobot().set_do(do = 6, state = 1)

In [35]:
target1 = Pose(
    position=Point(0.5, 0.4, 0.4),
    orientation=Quaternion(0.0, -0.7071067811865475, -0.7071067811865475, 0.0),
)
display_poses([target0, target1])
success = plan_goal_srv(Goal(pose=target1,
        vel_scale=0.2,
        acc_scale=0.2,
        planner='ptp'))


In [36]:
success = execute_trajectory_srv()
# UrRobot().set_do(4, False)
# UrRobot().set_do(5, True)